In [1]:
import pandas as pd
from itertools import combinations

In [2]:
core_df = pd.read_csv('bm_core.csv')
core_df.head()

,Авторы,Заглавие,Издание,Тип публикации
0,Корчак Владимир Юрьевич ...,Автоматизация планово-производственной деятель...,Компетентность\n \n 2016\n .-...,Статья в журнале
1,Верба Владимир Степанович ...,Авиационные комплексы радиолокационного дозора...,Информационно-измерительные и управляющие сист...,Статья в журнале
2,Зайцев Александр Вячеславович ...,Автоматизация поддержки решений при подготовке...,Известия высших учебных заведений. Машинострое...,Статья в журнале
3,Кравченко Виктор Филиппович ...,Автограф Виктора Петровича Шестопалова,Физические основы приборостроения\n \n ...,Статья в журнале
4,Майорова Вера Ивановна ...,Автоматизация приема и обработки резервной тел...,"Вестник МГТУ им.Н.Э.Баумана. Серия ""Машиностро...",Статья в журнале


In [4]:
has_scopus_name = lambda row: False if row['Издание'].find('Scopus') == -1 else True
core_df['Scopus'] = core_df.apply(has_scopus_name, axis=1)
core_df = core_df[core_df['Scopus']]

In [5]:
stucked_names = core_df['Авторы']
stucked_names.head()

66    Филиппов Сергей Сергеевич                     ...
28    Seregina E.V.                            \n(Ка...
30    Kristya V.I.                            \n(Кал...
31    Spiridonov I.N.                            \n(...
35    Белянчиков М.А.                            \n ...
Name: Авторы, dtype: object

In [6]:
def split_names(stucked_names_entry):
    splitted = stucked_names_entry.split('\n')
    str_list = []
    for substr in splitted:
        temp = ' '.join(substr.split())
        str_list.append(temp)
    str_list = list(filter(None, str_list))
    return str_list

In [7]:
def pairwise(t):
    it = iter(t)
    return zip(it,it)

In [8]:
def process_group_of_names(group_of_names):
    splitted = split_names(group_of_names)
    name_position_pairs = pairwise(splitted)
    bmstu_members = [entry for entry in name_position_pairs if (entry[1].find('Не МГТУ') == -1)]
    
    for entry in bmstu_members:
        personal_publications_num.setdefault(entry[0], 0)
        personal_publications_num[entry[0]] += 1
    for single_comb in combinations(bmstu_members, 2):
        sorted_comb = sorted(single_comb, key=lambda tup: tup[0])
        names_comb = (sorted_comb[0][0], sorted_comb[1][0])
        pair_publications_num.setdefault(names_comb, 0)
        pair_publications_num[names_comb] += 1

In [9]:
personal_publications_num = {}
pair_publications_num = {}
stucked_names = core_df['Авторы']
stucked_names.apply(process_group_of_names);

In [10]:
personal_publications_num_df = pd.DataFrame.from_dict(personal_publications_num, orient='index', columns=['pub_num'])
personal_publications_num_df.index.names = ['full_name']
personal_publications_num_df.reset_index(inplace=True)
personal_publications_num_df.loc[:, 'id'] = personal_publications_num_df.apply(lambda row: id(row['full_name']), axis=1)


pair_publications_num_df = pd.DataFrame.from_dict(pair_publications_num, orient='index', columns=['pair_pub_num'])
pair_publications_num_df.index.names = ['names_pair']
pair_publications_num_df.reset_index(inplace=True)
get_ids = lambda row: pd.Series([id(row['names_pair'][0]),
                                 id(row['names_pair'][1])])
pair_publications_num_df[['name1_id', 'name2_id']] = pair_publications_num_df.apply(get_ids, axis=1)

In [11]:
personal_publications_num_df

,full_name,pub_num,id
0,Филиппов Сергей Сергеевич,1,4665292080
1,Seregina E.V.,6,4610923952
2,Makarenkov A.M.,5,4610924016
3,Kristya V.I.,15,4610921328
4,Tun Y.N.,3,4610923760
5,Spiridonov I.N.,7,4611068144
6,Nekhina A.A.,1,4611068208
7,Горелик Владимир Семенович,24,4665292464
8,Пятышев А.Ю.,6,4664622072
9,Pomel'nikova A.S.,3,4665944368


In [12]:
pair_publications_num_df

,names_pair,pair_pub_num,name1_id,name2_id
0,"(Makarenkov A.M., Seregina E.V.)",4,4610924016,4610923952
1,"(Kristya V.I., Tun Y.N.)",3,4610921328,4610923760
2,"(Nekhina A.A., Spiridonov I.N.)",1,4611068208,4611068144
3,"(Горелик Владимир Семенович, Пятышев А.Ю.)",6,4665292464,4664622072
4,"(Bystrov N.S., Yatsenko P.I.)",1,4611068464,4611068720
5,"(Borodulin A.S., Nelyub V.A.)",4,4611068912,4611068016
6,"(Kobets L.P., Nelyub V.A.)",3,4611068784,4611068016
7,"(Malysheva G.V., Nelyub V.A.)",4,4611068976,4611068016
8,"(Borodulin A.S., Kobets L.P.)",8,4611068912,4611068784
9,"(Borodulin A.S., Malysheva G.V.)",9,4611068912,4611068976


In [38]:
personal_publications_num_df.to_json('pers_pub.json', orient='records')
pair_publications_num_df[['pair_pub_num', 'name1_id', 'name2_id']].to_json('pair_pub.json', orient='records')

In [15]:
personal_publications_num_network_format_df = personal_publications_num_df.rename(
    columns={'full_name':'title', 'pub_num':'value'})
pair_publications_num_network_format_df = pair_publications_num_df.rename(
    columns={'pair_pub_num':'value', 'name1_id':'from', 'name2_id':'to'})[['value', 'from', 'to']]

In [16]:
pair_publications_num_network_format_df.head()

,value,from,to
0,4,4610924016,4610923952
1,3,4610921328,4610923760
2,1,4611068208,4611068144
3,6,4665292464,4664622072
4,1,4611068464,4611068720


In [17]:
personal_publications_num_network_format_df.head()

,title,value,id
0,Филиппов Сергей Сергеевич,1,4665292080
1,Seregina E.V.,6,4610923952
2,Makarenkov A.M.,5,4610924016
3,Kristya V.I.,15,4610921328
4,Tun Y.N.,3,4610923760
